# Notebook to study short term persistence from multiple exposures in a single visit 


In [1]:
from astropy.io import fits
import glob, os, shutil, pickle, bz2, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sigmaclip
from scipy.optimize import curve_fit
from scipy.special import gammaincc, gamma
from astropy.wcs import WCS
from astropy.stats import histogram
from itertools import product
from multiprocessing import Pool

%matplotlib notebook

In [2]:
pwd

'/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/Pixel_based'

In [3]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'

In [4]:
# conversion factor from days to seconds
daytosec = 24.*3600.

In [5]:
#Single and double exponential models to be fitted to the data

def decay1(t,a1,t1):
    e1 = a1*np.exp(-t/t1)
    return e1

def intdec1(t,a1,t1):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td)
    
def decay2(t,a1,t1,a2,t2):
    e1 = a1*np.exp(-t/t1)
    e2 = a2*np.exp(-t/t2)
    return e1+e2

def intdec2(t,a1,t1,a2,t2):
    tu = t[1:]
    td = t[:-1]
    k1,k2  = -a1*t1, - a2*t2
    
    return k1*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) + k2*(np.exp(-tu/t2)-np.exp(-td/t2))/(tu-td)

#Single exponential models plus a constant

def intdec1_plusconst(t,a1,t1,q):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) +q

def dec1_plusconst(t,a1,t1,q):
    e1 = a1*np.exp(-t/t1)
    return e1+q


#Shifted power law model

def shpwl(t,t0,A,index):
    return A * ((t+t0)/1000)**index

def intshpwl(t,t0,A,index):
    tu = t[1:]
    td = t[:-1]

    if (index == -1.):
        return A*np.log( (tu+t0)/(td+t0) )
    else:
        return A/(1+index) * ( ((tu+t0)/1000)**(1+index) - ((td+t0)/1000)**(1+index) )/(tu-td)
    
    
#Schechter like model

def schechter(t,phi,alpha,tstar):
    x = t/tstar
    return phi*(x**alpha)*np.exp(-x)

def intschechter(t,phi,alpha,tstar):
    x = t/tstar

    tu = x[1:]
    td = x[:-1]

    g1 = gammaincc(alpha+1,td)
    g2 = gammaincc(alpha+1,tu)
    
    diff = gamma(alpha+1)*(g1-g2)
    
    return phi*diff


#Geometric median calculation function

from scipy.spatial.distance import cdist, euclidean

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1, D

        y = y1



In [6]:
# Define a function that takes the vsflts list, the current flt that is being used as stimulus
# and looks for all the pixels with valid stimulus values AND that have valid ramps (i.e. no source, only sky)
# in the following exposures AND that where not stimulated more than prior-stim-factor (psf)% of the current stimulus in 
# ANY past exposure up to a certain look back exposure (lb).
# Mario: added the option of multiplying the stimulus by the Pixel Area Map

def find_ramps(istim,flts,lev_u,lev_d,lb=None,PAM=None,psf=0.1):
        
    stimdata  = flts[istim,:,:]*(tendMJDs[istim]-tstrMJDs[istim])*daytosec
    
    if PAM is not None:
        stimdata *= PAM
    
    istimgood = (stimdata > lev_d) & (stimdata < lev_u)  
    print('Pixels with potentially right stimuli:',np.sum(istimgood) )

    if lb is not None:
        if (istim-lb)>0:
            st = istim-lb
        else:
            st = 0
    else:
        st = 0
    
    for i in range(st,istim,1):
        persdata = flts[i,:,:] * (tendMJDs[i]-tstrMJDs[i])*daytosec 
        if PAM is not None:
            persdata *= PAM

        if (imtyps[i] == 'EXT'):
            istimgood = istimgood & (persdata < psf*stimdata) 
            
    print('Pixels with really right stimuli:',np.sum(istimgood) )
    
    
    icount    = np.zeros_like(stimdata,dtype=np.int_)
    iprev     = istimgood
    for i in range(istim+1,len(imtyps),1):
    
        persdata = flts[i,:,:] * (tendMJDs[i]-tstrMJDs[i])*daytosec
        if PAM is not None:
            persdata *= PAM
    
        if (imtyps[i] == 'EXT'):
            msky = np.nanmean(sigmaclip(persdata,2.,2.)[0])
            ssky = np.nanstd(sigmaclip(persdata,2.,2.)[0])
            iskycurr = (persdata <msky+1*ssky) & (persdata >msky-1*ssky)    
        elif (imtyps[i] == 'DARK'):
            iskycurr = np.ones_like(persdata,dtype=np.bool_)
        else:
            print('Wrong image type')
            assert False
        
        igood = istimgood & iskycurr & iprev
        iprev = igood
        icount[igood] += 1

        print('Pixels with ramps extending for at least',i-istim,' exposures:', igood.sum())
        
        if (np.sum(igood) == 0):
            break
                 
    return icount

In [7]:
# Dedfine a function to get the sky value in the cureent flt pixel, but measured form the AD mosaic

def getskyfrommosaic(wcsAD, wcsFLT, x, y, dxgrid, dygrid, skyrad_o,skyrad_i,mask_sky_contam,mosaic):

    coords = wcsAD.all_world2pix(wcsFLT.all_pix2world(np.array([[x,y]],dtype=np.float_),0),0) 
    dx = coords[0,0]
    dy = coords[0,1]
    
    dst = np.sqrt((dxgrid-dx)**2 + (dygrid-dy)**2)
    msk = (dst<skyrad_o) & (dst > skyrad_i) & mask_sky_contam 
    skyarr = mosaic[1].data[msk]
    cskyarr,l,u = sigmaclip(skyarr,1.5,1.5)
    return np.nanmean(cskyarr)

# Similar but for getting background values from the smae image

def getlocalbackground (x, y, xgrid, ygrid, skyrad_o,skyrad_i, fltdata):

    dst = np.sqrt((xgrid-x)**2 + (ygrid-y)**2)
    msk = (dst<skyrad_o) & (dst > skyrad_i)
    skyarr = fltdata[msk]
    cskyarr,l,u = sigmaclip(skyarr,1.5,1.5)
    return np.nanmean(cskyarr)



In [8]:
def get_sky_and_indices(nz0, nz1, j, istim):
    # Function to get the sky value, as well as calculate the indices needed in the large data cube of IMA reads
       
    #Get the sky from the drizzled image
    imtyp = imtyps[istim+j]

    if(imtyp == 'EXT'):
        skyhere = getskyfrommosaic(w_mosaic, w_vsflts[istim+j], nz1, nz0, dxgrid, dygrid, skyrad_o,skyrad_i,mask_sky_contam,mosaic)
#        skyhere = getlocalbackground(nz1, nz0, xgrid, ygrid, skyrad_o,skyrad_i, flts[istim+j,:,:])
    elif(imtyp == 'DARK'):
        skyhere = 0.04 #Pleacholder, this is the mean dark current rate
    else:
        print('Wrong image type')
        assert False
    
    offset = ( tstrMJDs[istim+j] - tendMJDs[istim])*daytosec
    ioff = np.sum(nsamps[0:istim+j])
    nsamp = nsamps[istim+j]
    
    k_product = product([nz0], [nz1], [skyhere], [offset], [ioff], [nsamp], [j], range(nsamp-1))
    return list(k_product)

In [9]:
def get_pixel_values(inputs, istim, PAM=None):
    # function to extract the values from the IMA cube and IMA metadata arrays
    
    nz1, nz0, skyhere, offset, ioff, nsamp, j, k = inputs
    te = ima_times[ioff+k]
    ts = ima_times[ioff+k+1]

    tfromstim = te + offset
    tdenom    = te - ts  
    meancurr  = (ima_scis[ioff+k,nz1,nz0]*te - ima_scis[ioff+k+1,nz1,nz0]*ts)/tdenom
    stdvcurr  = np.sqrt(np.sum(np.square([ima_errs[ioff+k,nz1,nz0]*te,ima_errs[ioff+k+1,nz1,nz0]*ts])))/tdenom

    if ((PAM is not None) & (imtyps[istim+j] == 'EXT')):
        meancurr *= PAM[nz1,nz0]
        stdvcurr *= PAM[nz1,nz0]
        
    exptime = (tendMJDs[istim]-tstrMJDs[istim])*daytosec
    return [flts[istim,nz1,nz0]*exptime,exptime,nz0,nz1,tfromstim,tdenom,nsamp-k-1,nsamp,meancurr,stdvcurr,skyhere,istim,istim+j,imtyps[istim],imtyps[istim+j]]


In [10]:
#Read files header, make sure they are sorted by EXPSTART

sflts= []

for vis in ['1','2','3']:
    wdir = pdir+'/14016_data/Visit0'+vis+'/'
    flts = glob.glob(wdir+'*_flt.fits')
    print('***************')
    starttimes = []
    endtimes   = []
    imagetypes = []
    for flt in flts:
        starttimes.append(fits.getheader(flt,0)['EXPSTART'])
        endtimes.append(fits.getheader(flt,0)['EXPEND'])
        imagetypes.append(fits.getheader(flt,0)['IMAGETYP'])
        
    ii = np.argsort(starttimes)
    for jj in range(len(flts)):
        print(starttimes[ii[jj]],endtimes[ii[jj]],(-starttimes[ii[jj]]+endtimes[ii[jj]])*daytosec,imagetypes[ii[jj]],flts[ii[jj]][-18:])

    sflts.append([flts[i] for i in ii])

***************
57247.89789127 57247.90197646 352.96041641850024 EXT icrr01y7q_flt.fits
57247.9027409 57247.90682572 352.9284480493516 EXT icrr01y8q_flt.fits
57247.90759053 57247.91167535 352.9284480493516 EXT icrr01yaq_flt.fits
57247.91243979 57247.91652498 352.9604157898575 EXT icrr01ycq_flt.fits
57247.91728942 57247.9213746 352.9595520347357 EXT icrr01yeq_flt.fits
57247.92213905 57247.92622386 352.927583665587 EXT icrr01ygq_flt.fits
57247.92698868 57247.93107349 352.927583665587 EXT icrr01yiq_flt.fits
57247.93219683 57247.93913238 599.2315201554447 DARK icrr01ykq_flt.fits
57247.93937275 57247.94345794 352.9604157898575 DARK icrr01ymq_flt.fits
57247.94370164 57247.94778646 352.9284480493516 DARK icrr01yoq_flt.fits
57247.94803016 57247.95211535 352.96041641850024 DARK icrr01yqq_flt.fits
57247.95235905 57247.95644386 352.927583665587 DARK icrr01ysq_flt.fits
57247.95668757 57247.96077275 352.9595520347357 DARK icrr01yuq_flt.fits
57247.96125942 57247.96534423 352.927583665587 DARK icrr01

In [11]:
# Choose which visit to work on

visit_index = 0
vsflts = sflts[visit_index]

# If in a hurry, shorten the list for faster analysis

vsflts = vsflts

#plot all exposures multiple times for visualization of the selected pixels

sf=1.5
fig = plt.figure(figsize=(sf*len(vsflts),sf*len(vsflts)))

ax = []
for i,flt in enumerate(vsflts):
    im = fits.getdata(flt)
    c, low, upp = sigmaclip(im, 1.5,1.5)
    mn = np.mean(c)
    print(flt,'clipped mean: ',mn, 'AD sky:')
    j = -1
    while j < i: 
        j+=1
        ax.append(plt.subplot(len(vsflts),len(vsflts),j*len(vsflts)+1+i))
        ax[-1].imshow(np.log10(im/mn),cmap='viridis', interpolation='none', origin='lower')
        ax[-1].set_title(flt[-18:-9],fontsize=6)
        ax[-1].get_xaxis().set_ticks([])
        ax[-1].get_yaxis().set_ticks([])
        
        
# Read the mosaic file and plot it

plt.tight_layout()
mosaic = fits.open(mdir+'/F140W_Mosaic_WFC3_IR_drz.fits')

ax.append(plt.subplot(len(vsflts),len(vsflts),j*len(vsflts)+1))

pos1 = ax[-1].get_position() # get the original position 
pos2 = [pos1.x0, pos1.y0,  pos1.width * len(vsflts)/2., pos1.height * len(vsflts)/2.] 
ax[-1].set_position(pos2) # set a new position
ax[-1].get_xaxis().set_ticks([])
ax[-1].get_yaxis().set_ticks([])
        
c, low, upp = sigmaclip(mosaic[1].data[np.where(np.isfinite(mosaic[1].data))], 1.5,1.5)
mn = np.mean(c)
im = ax[-1].imshow(np.log10(mosaic[1].data/mn),cmap='viridis', interpolation='none', origin='lower')
ax[-1].set_title('Drizzled \n mosaic',fontsize=6)



<IPython.core.display.Javascript object>

/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01y7q_flt.fits clipped mean:  13.0913 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01y8q_flt.fits clipped mean:  12.8443 AD sky:


/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log10
/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in log10


/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yaq_flt.fits clipped mean:  12.7267 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01ycq_flt.fits clipped mean:  12.3651 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yeq_flt.fits clipped mean:  13.0769 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01ygq_flt.fits clipped mean:  11.9064 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yiq_flt.fits clipped mean:  11.0934 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01ykq_flt.fits clipped mean:  0.0211126 AD sky:
/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr0

/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log10
/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in log10


In [12]:
#Create the wcs objects for the AD mosaic and flts 

w_mosaic = WCS(mosaic[1].header)

w_vsflts = []
for vsflt in vsflts:
    w_vsflts.append(WCS(fits.getheader(vsflt,1)))

In [13]:
# Read in the pixel-area map

PAM = fits.getdata(pdir+'/Pixel_based/ir_wfc3_map.fits')

fig =plt.figure(figsize=(4,4))
plt.imshow(PAM)
plt.colorbar()

<IPython.core.display.Javascript object>

In [14]:
#From the current AD mosaic, get the sky values offsets that need to be used in the flts

flt2mosaic = list(mosaic[4].data['FILENAME']) #List of ALL the flts that contribute to the AD mosaic
MDRIZSKYs = []

for vsflt in vsflts:
    try:
        index_element = flt2mosaic.index(vsflt[-18:])
        MDRIZSKYs.append(mosaic[4].data['MDRIZSKY'][index_element])
    except ValueError:
        MDRIZSKYs.append(0.)

In [15]:
# Create the numpy arrays containg the ima and flt data as well
# as the arrays of metadata.
# Also subtract the MDRIZSKY from the flt for a 1-to-1 comaprison with the AD mosaic
#Also bring the darks into e/s

ima_scis  = []
ima_errs  = []
ima_times = [] 
flts      = []
tendMJDs  = []
tstrMJDs  = []
imtyps    = []
nsamps    = []
sampseqs  = []

for vsflt,MDS in zip(vsflts,MDRIZSKYs):
    print('Appending '+vsflt+' to the datacube')
    ima = fits.open(vsflt.replace('_flt','_ima'))
    nsamps.append(ima[0].header['NSAMP'])
    sampseqs.append(ima[0].header['SAMP_SEQ'])

    flt = fits.open(vsflt)
    if (flt[1].header['BUNIT'] == 'COUNTS/S'):
        fdt = flt[1].data*flt[0].header['CCDGAIN']
    elif (flt[1].header['BUNIT'] == 'ELECTRONS/S'):
        fdt = flt[1].data
    else:
        print('BUNITS not supported')
        assert False
    
    for k in range(nsamps[-1]):

        if (ima['SCI',k+1].header['BUNIT'] == 'COUNTS/S'):
            imas = ima['SCI',k+1].data[5:-5,5:-5]*ima[0].header['CCDGAIN']
            imae = ima['ERR',k+1].data[5:-5,5:-5]*ima[0].header['CCDGAIN']
            
        elif (ima['SCI',k+1].header['BUNIT'] == 'ELECTRONS/S'):
            imas = ima['SCI',k+1].data[5:-5,5:-5]
            imae = ima['ERR',k+1].data[5:-5,5:-5]
        else:
            print('BUNITS not supported')
            assert False

        ima_scis.append(imas)
        ima_errs.append(imae)
        ima_times.append(ima['TIME',k+1].header['PIXVALUE'])
   
    
    
    tendMJDs.append(flt[0].header['EXPEND'])
    tstrMJDs.append(flt[0].header['EXPSTART'])
    imtyps.append(flt[0].header['IMAGETYP'])
    flts.append(fdt - MDS)
    
    flt.close()
    ima.close()

print('Done1')
ima_scis  = np.asarray(ima_scis)
print('Done2')
ima_errs  = np.asarray(ima_errs)
print('Done3')
ima_times = np.asarray(ima_times)
print('Done4')
flts      = np.asarray(flts)
print('Done5')
tendMJDs  = np.asarray(tendMJDs)
print('Done6')
tstrMJDs  = np.asarray(tstrMJDs)
print('Done7')
imtyps    = np.asarray(imtyps)
print('Done8')
nsamps    = np.asarray(nsamps)
print('Done9')


Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01y7q_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01y8q_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yaq_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01ycq_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yeq_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01ygq_flt.fits to the datacube
Appending /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yiq_flt.fits to the d

In [16]:
#Define the stimuli e-/s level to identify the ramps

lev_u = np.inf
lev_d = 5e5

# Define the pixel grid (to trasform indices in x,y positions)
xgrid,ygrid = np.meshgrid( np.arange(fits.getdata(vsflts[0],1).shape[1]) ,np.arange(fits.getdata(vsflts[0],1).shape[0]))
dxgrid,dygrid = np.meshgrid( np.arange(mosaic[1].data.shape[1]) ,np.arange(mosaic[1].data.shape[0]))

drz_fin = np.isfinite(mosaic[1].data)

msky_d = np.nanmean(sigmaclip(mosaic[1].data[drz_fin],2.5,2.5)[0])
ssky_d = np.nanstd(sigmaclip(mosaic[1].data[drz_fin],2.5,2.5)[0])
mask_sky_contam = (mosaic[1].data <msky_d+3*ssky_d) & (mosaic[1].data >msky_d-3*ssky_d) & drz_fin

skyrad_o = 12
skyrad_i = 3
lookback = None
psf = 0.2
numcores = 8

/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  
/Users/gennaro/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [17]:
#before running the big step perform garbage collection
gc.collect()

df = pd.DataFrame()
cols = ['Stim','EXPTIME_stim','xpix','ypix','tfromstim','deltat','Read index','NSAMP','meancurr','stdvcurr','background','Ind_stim','Ind_pers','Stim_type','Pers_type']

mypool = Pool(numcores)

# Parallelized version
for istim,stim in enumerate(vsflts[:-1]):

    print('**********************')
    print('Doing: ',stim)

    if imtyps[istim] == 'EXT':
    
        icount    = find_ramps(istim,flts,lev_u,lev_d,lb=lookback,PAM=PAM,psf=psf)
    
        nz     = np.nonzero(icount)    
        nnexts = icount[nz]
    
        nlines = 0
        for nnext in nnexts:
            nlines = nlines+np.sum((nsamps-1)[istim+1:istim+1+nnext])

        print('Number of entries: ',nlines)
        modulus = np.trunc(nlines/10)
    
        if (nlines > 0) :
            flt_big_index = []
            for nz0,nz1,nnext in list(zip(nz[0],nz[1],nnexts)):
                prod = product([nz0], [nz1], range(1,nnext+1,1),[istim])
                flt_big_index += list(prod)
    
            derp = mypool.starmap(get_sky_and_indices, flt_big_index)
            ima_big_index = []
            for block in derp:
                ima_big_index += block
        
            biglist = mypool.starmap(get_pixel_values,zip(ima_big_index, [istim]*nlines, [PAM]*nlines))
            df = df.append(pd.DataFrame(biglist,columns=cols),ignore_index=True)
 

**********************
Doing:  /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01y7q_flt.fits
Pixels with potentially right stimuli: 27290
Pixels with really right stimuli: 27290
Pixels with ramps extending for at least 1  exposures: 962
Pixels with ramps extending for at least 2  exposures: 533
Pixels with ramps extending for at least 3  exposures: 366
Pixels with ramps extending for at least 4  exposures: 214
Pixels with ramps extending for at least 5  exposures: 125
Pixels with ramps extending for at least 6  exposures: 81
Pixels with ramps extending for at least 7  exposures: 81
Pixels with ramps extending for at least 8  exposures: 81
Pixels with ramps extending for at least 9  exposures: 81
Pixels with ramps extending for at least 10  exposures: 81
Pixels with ramps extending for at least 11  exposures: 81
Pixels with ramps extending for at least 12  exposures: 81
Pixels with ramps extending for at least 13  exposures: 81
Pix

**********************
Doing:  /user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence//14016_data/Visit01/icrr01yiq_flt.fits
Pixels with potentially right stimuli: 26390
Pixels with really right stimuli: 15067
Pixels with ramps extending for at least 1  exposures: 15067
Pixels with ramps extending for at least 2  exposures: 15067
Pixels with ramps extending for at least 3  exposures: 15067
Pixels with ramps extending for at least 4  exposures: 15067
Pixels with ramps extending for at least 5  exposures: 15067
Pixels with ramps extending for at least 6  exposures: 15067
Pixels with ramps extending for at least 7  exposures: 15067
Pixels with ramps extending for at least 8  exposures: 15067
Pixels with ramps extending for at least 9  exposures: 15067
Pixels with ramps extending for at least 10  exposures: 15067
Pixels with ramps extending for at least 11  exposures: 15067
Pixels with ramps extending for at least 12  exposures: 15067
Pixels with ramps extending for

In [18]:
with bz2.BZ2File(sdir+'Test_DF.pbz2', 'w') as f:
        pickle.dump(df, f)

In [ ]:
print('Total number of entries',len(df))
df.head()

Total number of entries 4191843


,Stim,EXPTIME_stim,xpix,ypix,tfromstim,deltat,Read index,NSAMP,meancurr,stdvcurr,background,Ind_stim,Ind_pers,Stim_type,Pers_type
0,804955.269599,352.960416,552,0,418.987126,25.00051,15,16,8.553245,3.795684,9.199046,0,1,EXT,EXT
1,804955.269599,352.960416,552,0,393.986616,25.00052,14,16,9.190909,3.687510,9.199046,0,1,EXT,EXT
2,804955.269599,352.960416,552,0,368.986096,25.00052,13,16,8.901025,3.573037,9.199046,0,1,EXT,EXT
3,804955.269599,352.960416,552,0,343.985576,25.00052,12,16,10.871115,3.445416,9.199046,0,1,EXT,EXT
4,804955.269599,352.960416,552,0,318.985056,25.00052,11,16,9.208886,3.310966,9.199046,0,1,EXT,EXT


In [ ]:
gBMs    =   [np.ones(len(df),dtype=np.bool_)]
gBMs.append( (df['Ind_pers'] != 7) )
gBMs.append( (df['Ind_pers'] != 7) & (df['Read index'] != 1) & (df['Ind_stim'] != 6) )
gBMs.append( (df['Ind_pers'] == 7) & (df['deltat'] > 10 ))
gBMs.append( (df['Ind_pers'] > 7)  & (df['Read index'] != 1) )
gBMs.append( (df['Ind_pers'] < 7)  & (df['Read index'] != 1) )
gBMs.append( (df['Ind_pers'] != 7) & (df['Read index'] != 1) )

fig = plt.figure(figsize=(15,4*len(gBMs)))
ax = []

for i,gBM in enumerate(gBMs):

    df2 = df[gBM]

    ax.append(fig.add_subplot(len(gBMs),2,1+i*2))
    ax.append(fig.add_subplot(len(gBMs),2,2+i*2))
    
    col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))

    ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
    ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

    step = 25.
    tmin = np.min(df2['tfromstim'].values)
    tmax = np.max(df2['tfromstim'].values)

    tmed   = []
    medsig = []

    while tmin <= tmax:

        BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
        tmin += step
        if (np.sum(BM) > 5):
            medsig.append(np.nanmedian(df2['meancurr'][BM].values-df2['background'][BM].values))
            tmed.append(np.median(df2['tfromstim'][BM].values))
    
            #gv = sigmaclip((df2['meancurr'][BM].values-df2['background'][BM].values),2.5,2.5)[0]
            #if (len(gv) >= 3):
                #medsig.append(np.nanmedian(gv))
                #tmed.append(np.median(df2['tfromstim'][BM].values))
    
    ax[-2].plot(tmed,medsig,c='r')
    ax[-1].plot(tmed,medsig,c='r')               
    ax[-1].plot(np.array([1,10000]),700*np.array([1.,1/10000.]),c='black')

    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

<IPython.core.display.Javascript object>

In [ ]:
gstims = np.unique(df['Ind_stim'] )

fig = plt.figure(figsize=(15,4*gstims.size))

ax = []

for j,i in enumerate(gstims):
    gBMh = (df['Ind_stim'] == i) & (df['Ind_pers'] != 7) & (df['Read index'] != 1) &  (df['Ind_pers'] <7)
    
    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]
        ax.append(fig.add_subplot(gstims.size,2,j*2+1))
        ax.append(fig.add_subplot(gstims.size,2,j*2+2))

        col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

        tmedh = []
        medsigh= []
        
        step = 25
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
 
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

In [ ]:
gstims = np.unique(df['Ind_stim'] )

fig = plt.figure(figsize=(15,4*gstims.size))

ax = []

for j,i in enumerate(gstims):
    gBMh = (df['Ind_stim'] == i) & (df['Ind_pers'] != 7) & (df['Read index'] != 1) &  (df['Ind_pers'] >7)
    
    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]
        ax.append(fig.add_subplot(gstims.size,2,j*2+1))
        ax.append(fig.add_subplot(gstims.size,2,j*2+2))

        col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

        tmedh = []
        medsigh= []
        
        step = 25
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
  
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

In [ ]:
mf=1.2
levels = lev_d * np.array([1,np.power(mf,1),np.power(mf,2),np.power(mf,3),np.power(mf,4),np.power(mf,5),np.power(mf,6),np.power(mf,7)])

fig = plt.figure(figsize=(15,4*levels.size))

ax = []

colors = np.array(['#e41a1c', '#377eb8', '#4daf4a','#a44a7c','#123321','#cad111','#111cad','#abccdb','#dd88a1','#ab0011',
                   '#aa11ca','#b18522','#097023','#909090','#090909','#FDEEDF','DDDDDD','#556611','#0911AA','#8FD45A'])

for j,(low,high) in enumerate(zip(list(levels[:-1]),list(levels[1:]))):
    gBMh = (df['Ind_pers'] != 7) & (df['Read index'] != 1) & (df['Stim'] >= low) & (df['Stim']<high)

    ax.append(fig.add_subplot(levels.size,2,j*2+1))
    ax.append(fig.add_subplot(levels.size,2,j*2+2))

    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]

        ax[-2].set_title('Low:'+str(low)+' High:'+str(high)+' Np:'+str(np.sum(gBMh)))
        ax[-1].set_title('Low:'+str(low)+' High:'+str(high)+' Np:'+str(np.sum(gBMh)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=[df2['Ind_stim'].values])
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=[df2['Ind_stim'].values])

        tmedh = []
        medsigh= []
        
        step = 50
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
  
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()